In [1]:
import rdkit

import pandas as pd
import numpy as np
from copy import deepcopy

from torch_geometric.datasets import MoleculeNet
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import AllChem

In [2]:
import custom_classification_functions
import custom_models
from custom_classification_functions import *
from custom_models import *
import custom_general_functions
from custom_general_functions import *

from copy import deepcopy

In [3]:
from torch_geometric.nn import global_mean_pool, global_max_pool

model_type = "GNN"

filename_basic = "results_classification/model_gnn_conv"

apply_scaffold_split = False

hidden_channels=[64]
gcn_layers=4
linear_sizes=[[512], [512, 256]]
aggregations=[global_mean_pool, global_max_pool]
apply_random_aggregations=False

learning_rate=0.001

In [4]:
param_combinations = dict()

hidden_channels_list = [[64], [64, 128]]
linear_sizes_list = [[], [512], [512, 256]]
gcn_layers_list = [1, 2, 3, 4]
aggregations_list = [[global_mean_pool, global_max_pool], [global_mean_pool], [global_max_pool]]
apply_scaffold_split_list = [False, True]

In [5]:
create_new_data = True

In [6]:
data, data_y = load_hiv()

pytorch_graph_list = create_pytorch_graph(data)

processed_data = process_pytorch_graph(pytorch_graph_list, data_y)

train_dataset, test_dataset, val_dataset = create_train_test_graphs(processed_data, train_percentage= 0.8, test_percentage=0.1, apply_scaffold_split = False )
loader, test_loader, val_loader = create_dataloader_val(train_dataset, test_dataset, val_dataset, batch_size=64)

j:\Anaconda3\envs\newest_dgl\lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [7]:
train_dataset, test_dataset, val_dataset 

(<torch.utils.data.dataset.Subset at 0x178330fc070>,
 <torch.utils.data.dataset.Subset at 0x1783551f0d0>)

In [8]:
datasets = [train_dataset, test_dataset, val_dataset ]

In [9]:
loaders = [loader, test_loader, val_loader]

In [10]:
import torch_geometric.data

In [11]:
from torch_geometric.data import *

In [12]:
train_dataset.dataset

[Data(x=[19, 9], edge_index=[2, 40], edge_attr=[40, 3], smiles='CCC1=[O+][Cu-3]2([O+]=C(CC)C1)[O+]=C(CC)CC(CC)=[O+]2', y=[1, 1]),
 Data(x=[39, 9], edge_index=[2, 88], edge_attr=[88, 3], smiles='C(=Cc1ccccc1)C1=[O+][Cu-3]2([O+]=C(C=Cc3ccccc3)CC(c3ccccc3)=[O+]2)[O+]=C(c2ccccc2)C1', y=[1, 1]),
 Data(x=[21, 9], edge_index=[2, 48], edge_attr=[48, 3], smiles='CC(=O)N1c2ccccc2Sc2c1ccc1ccccc21', y=[1, 1]),
 Data(x=[24, 9], edge_index=[2, 50], edge_attr=[50, 3], smiles='Nc1ccc(C=Cc2ccc(N)cc2S(=O)(=O)O)c(S(=O)(=O)O)c1', y=[1, 1]),
 Data(x=[10, 9], edge_index=[2, 18], edge_attr=[18, 3], smiles='O=S(=O)(O)CCS(=O)(=O)O', y=[1, 1]),
 Data(x=[16, 9], edge_index=[2, 32], edge_attr=[32, 3], smiles='CCOP(=O)(Nc1cccc(Cl)c1)OCC', y=[1, 1]),
 Data(x=[10, 9], edge_index=[2, 20], edge_attr=[20, 3], smiles='O=C(O)c1ccccc1O', y=[1, 1]),
 Data(x=[18, 9], edge_index=[2, 38], edge_attr=[38, 3], smiles='CC1=C2C(=COC(C)C2C)C(O)=C(C(=O)O)C1=O', y=[1, 1]),
 Data(x=[26, 9], edge_index=[2, 54], edge_attr=[54, 3], smile

In [13]:
len(train_dataset)

32901

In [14]:
len(test_dataset)

4112

In [15]:
test_dataset.dataset

[Data(x=[19, 9], edge_index=[2, 40], edge_attr=[40, 3], smiles='CCC1=[O+][Cu-3]2([O+]=C(CC)C1)[O+]=C(CC)CC(CC)=[O+]2', y=[1, 1]),
 Data(x=[39, 9], edge_index=[2, 88], edge_attr=[88, 3], smiles='C(=Cc1ccccc1)C1=[O+][Cu-3]2([O+]=C(C=Cc3ccccc3)CC(c3ccccc3)=[O+]2)[O+]=C(c2ccccc2)C1', y=[1, 1]),
 Data(x=[21, 9], edge_index=[2, 48], edge_attr=[48, 3], smiles='CC(=O)N1c2ccccc2Sc2c1ccc1ccccc21', y=[1, 1]),
 Data(x=[24, 9], edge_index=[2, 50], edge_attr=[50, 3], smiles='Nc1ccc(C=Cc2ccc(N)cc2S(=O)(=O)O)c(S(=O)(=O)O)c1', y=[1, 1]),
 Data(x=[10, 9], edge_index=[2, 18], edge_attr=[18, 3], smiles='O=S(=O)(O)CCS(=O)(=O)O', y=[1, 1]),
 Data(x=[16, 9], edge_index=[2, 32], edge_attr=[32, 3], smiles='CCOP(=O)(Nc1cccc(Cl)c1)OCC', y=[1, 1]),
 Data(x=[10, 9], edge_index=[2, 20], edge_attr=[20, 3], smiles='O=C(O)c1ccccc1O', y=[1, 1]),
 Data(x=[18, 9], edge_index=[2, 38], edge_attr=[38, 3], smiles='CC1=C2C(=COC(C)C2C)C(O)=C(C(=O)O)C1=O', y=[1, 1]),
 Data(x=[26, 9], edge_index=[2, 54], edge_attr=[54, 3], smile

In [16]:
len(test_dataset.dataset)

41127

In [17]:
len(test_dataset)

4112

In [18]:
import dill as pickle
with open('datasets_class_hiv.pkl', 'wb') as file: 
      
    # A new file will be created 
    pickle.dump(datasets, file) 

In [19]:
data, data_y = load_hiv()

pytorch_graph_list = create_pytorch_graph(data)

processed_data = process_pytorch_graph(pytorch_graph_list, data_y)

train_dataset, test_dataset, val_dataset = create_train_test_graphs(processed_data, train_percentage= 0.8, test_percentage=0.1, apply_scaffold_split = True )
loader, test_loader, val_loader = create_dataloader_val(train_dataset, test_dataset, val_dataset, batch_size=64)

datasets_scaffold = [train_dataset, test_dataset, val_dataset ]

loaders_scaffold = [loader, test_loader, val_loader]
import dill as pickle
with open('datasets_scaffold_class_hiv.pkl', 'wb') as file: 
      
    # A new file will be created 
    pickle.dump(datasets_scaffold, file) 

Start initializing RDKit molecule instances...
Creating RDKit molecule instance 1000/41127
Creating RDKit molecule instance 2000/41127
Creating RDKit molecule instance 3000/41127
Creating RDKit molecule instance 4000/41127
Creating RDKit molecule instance 5000/41127
Creating RDKit molecule instance 6000/41127
Creating RDKit molecule instance 7000/41127
Creating RDKit molecule instance 8000/41127
Creating RDKit molecule instance 9000/41127
Creating RDKit molecule instance 10000/41127
Creating RDKit molecule instance 11000/41127
Creating RDKit molecule instance 12000/41127
Creating RDKit molecule instance 13000/41127
Creating RDKit molecule instance 14000/41127
Creating RDKit molecule instance 15000/41127
Creating RDKit molecule instance 16000/41127
Creating RDKit molecule instance 17000/41127
Creating RDKit molecule instance 18000/41127
Creating RDKit molecule instance 19000/41127
Creating RDKit molecule instance 20000/41127
Creating RDKit molecule instance 21000/41127
Creating RDKit mo

j:\Anaconda3\envs\newest_dgl\lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
